## BIOINF529 Exam #1 - Winter 2019
This exam is worth 15% of your final grade.

The exam is due 1 week after assigned as enforced by Canvas.<br>
By turning in this exam you are indicating your acceptance of this statement: “I pledge that this examination is solely my own work.” <br>
You may not consult or collaborate with others. All answers must be your own. <br>
You are allowed to ask questions at office hours but the answers given will be high-level/conceptual in nature. <br>

Please rename this notebook to **exam1_uniqname.ipynb** for submission!<br>

Topic areas: motif finding, MCMC, HMMs<br>


### 1. Nth order Markov chain
We previously discussed and implemented a 1st order Markov chain for processing words and generating random sentences from Dr. Seuss. In this model, each event or word is output from only the previous state with no memory of any prior states. While this is useful in some cases, typical biological applications of Markov chains require higher-order models to accurately capture what we know about a system. For instance, in attempting to identify coding regions of a genome, we know that open reading frames (ORFs) contain codon triplets, and so a third or sixth order Markov chain would better describe these regions. In this exam, you will implement a generalized form of our previous Markov Chain to allow for Nth order chains.

As in class, please provide at least two functions (additional helper functions are left to your discretion) that will provide the following:
- build_markov_model(markov_model, text, order=N): as in class, takes as input an existing markov model and updates it with 'text'. The updates now will be as a Nth order rather than a 1st order model.
- generate_random_text(markov_model): as in class this function returns a randomly generated series of states from the Markov movel.


In [ ]:
# Please implement all of your functions for question 1 below. The two functions described below are required.

def build_markov_model(markov_model, text, order=1):
    '''
    Function to build or add to a Nth order Markov model given a string of text

    Args: 
        markov_model (dict of dicts): a dictionary of word:(next_word:frequency pairs)
            or None if a new model is being built
        new_text (str): a string to build or add to the moarkov_model
        order (int): the number of previous states to consider for the model
        
    Returns:
        markov_model (dict of dicts): an updated/new markov_model
    '''
    
def generate_random_text(markov_model):
    '''
    Function to generate random text given a markov model
    
    Args: 
        markov_model (dict of dicts): a dictionary of word:(next_word:frequency pairs)

    Returns:
        sentence (str): a randomly generated sequence given the model
    '''


In [ ]:
#Example testing of the above implementation:
text = "One fish two fish red fish blue fish"
markov_model = build_markov_model(markov_model, text, 1)
print (markov_model)


### 2. Baum-Welch
In class we have now implemented aspects of evaluating a hidden Markov model through Viterbi and the Forward, Backward, and Forward-Backward algorithms. Each of these algorithms requires prior knowledge of dataset labels in order to generate the state inititaion, transition, and emission probabilities. You will now implement the Baum-Welch algorithm which will learn these probabilities for your model in an unsupervised manner. For this assignement, you may use any of the previously provided class implementations as part of your submission.

Initialization: <br>
Set arbitrary parameters for $\Theta$ in $A$ transition probabilities, $E$ emission probabilities, and $B$ starting probabilities. Alternatively, set arbitrary state labels $\pi$ and use these to estimate parameters.

Expectation:<br>
For each input sequence $x$ where sequence index $j = 1 \dots n $:<br>
Calculate $f_{k}(i)$ matrix for sequence $x$ using the Forward algorithm.<br>
Calculate $r_{k}(i)$ matrix for sequence $x$ using the Backward algorithm.<br>
Update transition matrix $A_{kl}$ by summing over all positions ($i=1\dots T-1$):<br>
$A_{kl} = \sum_{j}1/P(x^{j}) \sum_{i}f_{k}^{j}(i)a_{kl}e_{l}(x_{i+1}^{j})r_{l}^{j}(i+1)$ <br>
where $x^{j}$, $f^{j}$, and $r^{j}$ are sequence, forward matrix, and backward matrix for squence index $j$ respectively.<br>
Update emission matrix $E_{k}$ by summing over all positions ($i=1\dots T$):<br>
$E_{k}(\sigma) = \sum_{j}1/P(x^{j}) \sum_{i|x_{i}^{j}=\sigma}f_{k}^{j}(i)r_{k}^{j}(i)$ <br>
where the inner sum is only over positions $i$ that have emission $\sigma$. <br>
Update initial state matrix:<br>
$B_{k} = \sum_{j}1/P(x^{j}) \sum_{k}f_{k}^{j}(0)r_{k}^{j}(0)$

Maximization:<br>
Calculate new model parameters as we did with Markov Chains:<br>
$a_{kl} = A_{kl}/\sum_{l}A_{kl}$<br>
$e_{k}(\sigma) = E_{k}(\sigma) / \sum_{\sigma}E_{k}(\sigma)$<br>
$b_{k} = B_{k} / \sum_{k}{B_k}$

<Br>Termination: <br>
Stop at convergence as measured by log likelihood or is maximum number of iterations has been reached.

Please add to the HMM class a new function baum_welch(self, sequences, pseudocount=1e-100) that takes as input a list of sequences to train the model. 
    
I have added implementations of associated functions below, but you may use your own implementations from class.

In [ ]:
import json

class HMM(object):
    """Main class for HMM objects
    
    Class for holding HMM parameters and to allow for implementation of
    functions associated with HMMs
    
    Private Attributes:
        _alphabet (set): The alphabet of emissions
        _hidden_states (set): Hidden states in the model
        _transitions (dict(dict)): A dictionary of transition probabilities
        _emissions (dict(dict)): A dictionary of emission probabilities
        _initial (dict): A dictionary of initial state probabilities

    """

    __all__ = ['viterbi', 'forward', 'backward', 'forward_backward', 'baum-welch']

    def __init__(self, alphabet, hidden_states, A=None, E=None, B=None, seed=None):
        self._alphabet = set(alphabet)
        self._hidden_states = set(hidden_states)
        self._transitions = A
        self._emissions = E
        self._initial = B
        self._seed = seed
        if(self._transitions == None):
            self._initialize_random(self._alphabet, self._hidden_states, self._seed)
            
    def __str__(self):
        out_text = [f'Alphabet: {self._alphabet}',
                    f'Hidden States: {self._hidden_states}',
                    f'Initial Probabilities: {json.dumps(self._initial, sort_keys = True, indent=4)}',
                    f'Transition Probabilities: {json.dumps(self._transitions, sort_keys = True, indent=4)}',
                    f'Emission Probabilities: {json.dumps(self._emissions, sort_keys = True, indent=4)}']
        return '\n'.join(out_text)
    
    @classmethod
    def __dir__(cls):
        return cls.__all__
    
    def _emit(self, cur_state, symbol):
        return self._emissions[cur_state][symbol]
    
    def _transition(self, cur_state, next_state):
        return self._transitions[cur_state][next_state]
    
    def _init(self, cur_state):
        return self._initial[cur_state]

    def _states(self):
        for k in self._hidden_states:
            yield k
    
    
    def _get_alphabet(self):
        for sigma in self._alphabet:
            yield sigma
            
    def _initialize_random(self, alphabet, states, seed):
        alphabet = list(set(alphabet))
        alphabet.sort()
        states = list(set(states))
        states.sort()
        self._alphabet = alphabet
        self._hidden_states = states

        #Initialize empty matrices A and E with pseudocounts
        A = {}
        E = {}
        I = {}
        np.random.seed(seed=seed)
        I_rand = np.random.dirichlet(np.ones(len(self._hidden_states)))
        for i, state in enumerate(self._states()):
            E[state] = {}
            A[state] = {}
            I[state] = I_rand[i]
            E_rand = np.random.dirichlet(np.ones(len(self._alphabet)))
            A_rand = np.random.dirichlet(np.ones(len(self._hidden_states)))
            for j, sigma in enumerate(self._get_alphabet()):
                E[state][sigma] = E_rand[j]
            for j, next_state in enumerate(self._states()):
                A[state][next_state] = A_rand[j]
                
        self._transitions = A
        self._emissions = E
        self._initial = I
        return
    
    def viterbi(self, sequence):
        """ The viterbi algorithm for decoding a string using a HMM

        Args:
            sequence (list): a list of valid emissions from the HMM

        Returns:
            result (list): optimal path through HMM given the model parameters
                           using the Viterbi algorithm
        
        Pseudocode for Viterbi:
            Initialization (𝑖=0): 𝑣𝑘(𝑖)=𝑒𝑘(𝜎)𝑏𝑘.
            Recursion (𝑖=1…𝑇): 𝑣𝑙(𝑖)=𝑒𝑙(𝑥𝑖) max𝑘(𝑣𝑘(𝑖−1)𝑎𝑘𝑙); 
                                ptr𝑖(𝑙)= argmax𝑘(𝑣𝑘(𝑖−1)𝑎𝑘𝑙).
            Termination: 𝑃(𝑥,𝜋∗)= max𝑘(𝑣𝑘(𝑙)𝑎𝑘0); 
                             𝜋∗𝑙= argmax𝑘(𝑣𝑘(𝑙)𝑎𝑘0).
            Traceback: (𝑖=𝑇…1): 𝜋∗𝑖−1= ptr𝑖(𝜋∗𝑖).
        """

        # Initialization (𝑖=0): 𝑣𝑘(𝑖)=𝑒𝑘(𝜎)𝑏𝑘.
        # Initialize trellis and traceback matrices
        # trellis will hold the vi data as defined by Durbin et al.
        # and trackback will hold back pointers (19)
        trellis = {} # This only needs to keep the previous column probabilities
        traceback = [] # This will need to hold all of the traceback data so will be an array of dicts()
        for state in self._states():
            trellis[state] = np.log10(self._init(state)) + np.log10(self._emit(state, sequence[0])) # b * e(0) for all k
            
        # Next we do the recursion step:
        # Recursion (𝑖=1…𝑇): 𝑣𝑙(𝑖)=𝑒𝑙(𝑥𝑖) max𝑘(𝑣𝑘(𝑖−1)𝑎𝑘𝑙); 
        #                 ptr𝑖(𝑙)= argmax𝑘(𝑣𝑘(𝑖−1)𝑎𝑘𝑙).
        for t in range(1, len(sequence)):  # For each position in the sequence
            trellis_next = {}
            traceback_next = {}

            for next_state in self._states():    # Calculate maxk and argmaxk
                k={}
                for cur_state in self._states():
                    k[cur_state] = trellis[cur_state] + np.log10(self._transition(cur_state, next_state)) # k(t-1) * a
                argmaxk = max(k, key=k.get)
                trellis_next[next_state] =  np.log10(self._emit(next_state, sequence[t])) + k[argmaxk] # k * e(t)
                traceback_next[next_state] = argmaxk
                
            #Overwrite trellis 
            trellis = trellis_next
            #Keep trackback pointer matrix
            traceback.append(traceback_next)
            
        # Termination: 𝑃(𝑥,𝜋∗)= max𝑘(𝑣𝑘(𝑙)𝑎𝑘0); 
        #                  𝜋∗𝑙= argmax𝑘(𝑣𝑘(𝑙)𝑎𝑘0).
        max_final_state = max(trellis, key=trellis.get)
        max_final_prob = trellis[max_final_state]
                
        # Traceback: (𝑖=𝑇…1): 𝜋∗𝑖−1= ptr𝑖(𝜋∗𝑖).
        result = [max_final_state]
        for t in reversed(range(len(sequence)-1)):
            result.append(traceback[t][max_final_state])
            max_final_state = traceback[t][max_final_state]

        return result[::-1]

    def forward(self, sequence):
        """ The forward algorithm for calculating probability of sequence given HMM

        Args:
            sequence (list): a list of valid emissions from the HMM

        Returns:
            result (float, list of dicts): P(x) and the f matrix as a list
        
        Pseudocode for Forward:
            Initialization (𝑖=0): 𝑓𝑘(0)=𝑒𝑘(𝜎0)𝑏𝑘.
            Recursion (𝑖=1…𝑇): 𝑓𝑙(𝑖)=𝑒𝑙(𝜎𝑖)∑𝑘(𝑓𝑘(𝑖−1)𝑎𝑘𝑙)
            Termination: 𝑃(𝑥)=∑𝑘𝑓𝑘(𝑇)
        """

        # Initialization (𝑖=0): 𝑓𝑘(0)=𝑒𝑘(𝜎0)𝑏𝑘.
        # Initialize f
        f = [] # For this algorithm it is helpful to keep this entire matrix
        f.append({})
        for state in self._states():
            f[-1][state] = self._init(state) * self._emit(state, sequence[0]) # b * e(0) for all k

        # Next we do the recursion step:
        # Recursion (𝑖=1…𝑇): 𝑓𝑙(𝑖)=𝑒𝑙(𝜎𝑖)∑𝑘(𝑓𝑘(𝑖−1)𝑎𝑘𝑙) 
        for i in range(1, len(sequence)):  # For each position in the sequence
            f.append({})
            for next_state in self._states(): # For each state
                f[-1][next_state] = 0
                for cur_state in self._states():
                    f[-1][next_state] += f[i-1][cur_state] * self._transition(cur_state, next_state) # sum of f(i-1) * a
                f[-1][next_state] =  self._emit(next_state, sequence[i]) * f[-1][next_state] # f * e(i)
        
        # Termination: 𝑃(𝑥)=∑𝑘𝑓𝑘(𝑇)
        Px = 0
        for state in self._states():
            Px += f[-1][state]
            
        return Px, f

    def backward(self, sequence):
        """ The backward algorithm for calculating probability of sequence given HMM

        Args:
            sequence (list): a list of valid emissions from the HMM

        Returns:
            result (float, list of dicts): P(x) and the b matrix as a list
        
        Pseudocode for Backward:
            Initialization (𝑖=T): 𝑟𝑘(𝑇)=1.
            Recursion (𝑖=𝑇−1…1): 𝑟𝑘(𝑖)=∑𝑙𝑟𝑙(𝑖+1)𝑎𝑘𝑙𝑒𝑙(𝜎𝑖+1)
            Termination: 𝑃(𝑥)=∑𝑙𝑟𝑘(1)𝑒𝑙(𝜎1)𝑏𝑙
        """

        # Initialization (𝑖=T): 𝑟𝑘(𝑇)=1.
        # Initialize r
        r = [] # For this algorithm it is helpful to keep this entire matrix
        r.insert(0, {})
        for state in self._states():
            r[0][state] = 1 # 1 for all k

        # Next we do the recursion step:
        # Recursion (𝑖=T-1…1): 𝑟𝑘(𝑖)=∑𝑙𝑟𝑙(𝑖+1)𝑎𝑘𝑙𝑒𝑙(𝜎𝑖+1)
        for i in range(len(sequence)-1, 0, -1):  # For each position in the sequence in reverse
            r.insert(0, {}) # append a new item at the beginning
            for prev_state in self._states(): # For each state
                r[0][prev_state] = 0
                for next_state in self._states():
                    r[0][prev_state] += r[1][next_state] * self._transition(prev_state, next_state) * self._emit(next_state, sequence[i])

        # Termination: 𝑃(𝑥)=∑𝑙𝑟𝑘(1)𝑒𝑙(𝜎1)𝑏𝑙
        Px = 0
        for state in self._states():
            Px += r[0][state] * self._init(state) * self._emit(state, sequence[0])
                        
        return Px, r
    
    def forward_backward(self, sequence):
        """ The forward-backward algorithm for calculating marginal posteriors given HMM

        Args:
            sequence (list): a list of valid emissions from the HMM

        Returns:
            posterior (list of dicts): all posteriors as a list
        
        Pseudocode for Forward-Backward:
            Calculate f[] as forward algorithm
            Calculate r[] as backward algorithm
            for all i in sequence
                for all states
                    posterior[i][state] = f[i][state] * r[i][state] / Px
        """        
        #Calculate forward and backward matrices
        f_Px, f_matrix = self.forward(sequence)
        r_Px, r_matrix = self.backward(sequence)
    
        posterior = []
        for i in range(0, len(sequence)):  # For each position in the sequence
            posterior.append({})
            for state in self._states(): # For each state
                posterior[i][state] = f_matrix[i][state] * r_matrix[i][state] / f_Px
                
        return posterior
    
    def baum_welch(self, sequences, pseudocount=1e-100):
        """ The baum-welch algorithm for unsupervised HMM parameter learning

        Args:
            sequence (list): a list of sequences containing valid emissions from the HMM
            pseudocount (float): small pseudocount value (default: 1e-100)

        Returns:
            None but updates the current HMM model parameters:
             self._transitions, self._emissions, self._initial
        
        """     



        

In [ ]:
# This section of code will initialize your HMM

hidden_states = ('I', 'G') # CpG Island or Genome
alphabet = ('A', 'C', 'G', 'T') # DNA Alphabet

model = HMM(alphabet, hidden_states, seed=70)

sequence = "ACGCGATCATACTATATTAGCTAAATAGATACGCGCGCGCGCGCGATATATATATATAGCTAATGATCGATTACCCCCCCCCCCAATTA"

print(model)

In [ ]:
# Example testing of the above implementation
model.baum_welch([sequence])
print(model)